In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys, copy
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.45,                 # Elongation factor
    delta       = 0.35,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04                  # position of the LCFS (x_inner)
)

simulation.set_data_param(
    expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/',
    g0simdir   = home_dir+'/g0_sim/',
    simname    = '48x32x16x16x6_nunifvg',
    simdir     = 'gk_tcv_posD_iwl_3x2v_AH/',
    fileprefix = 'gk_tcv_posD_iwl_3x2v_D02',
    wkdir      = 'wk/'
)

# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)
# Check available frames
tfs = find_available_frames(simulation,'field')
print(tfs)

#---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','LCFS')
simulation.normalize('y','Larmor radius')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')

In [ ]:
#---- Compute particle loss due to fluctuations over time in the simulation
# (this does not depend on the srcGB parameters)
twindow = tfs
GBloss_i_t, time = simulation.get_GBloss_t(
    species=ion,
    twindow=twindow,
    ix=0)

GBloss_e_t, time = simulation.get_GBloss_t(
    species=elc,
    twindow=twindow,
    ix=0)

In [ ]:
#---- Evaluate the particle input from the a priori numerical gradB source
simulation.set_GBsource(    
    n_srcGB     = 3.185e23,
    T_srcGB     = None,
    x_srcGB     = 0,
    sigma_srcGB = 10*simulation.get_rho_s(),
    bfac_srcGB  = 1.4,
    temp_model  = 'constant',
    dens_model  = 'singaus'
)
# Store the old version of GBsource which has an a priori amplitude
GBsource_apriori = simulation.GBsource

# Evaluate the artificial gradB source
[x,y,z] = simulation.geom_param.get_conf_grid()
[X,Y,Z] = custom_meshgrid(x,y,z)
GBs_xyz = GBsource_apriori.dens_source(X,Y,Z)*simulation.geom_param.Jacobian
GBs_tot = integral_xyz(x,y,z,GBs_xyz)
GBsrc_t = [GBs_tot for t_ in time]


In [ ]:
eta2 = simulation.geom_param.GBflux_model(1.2)
eta22 = simulation.geom_param.GBflux_model(1.5)
[x,y,z] = simulation.geom_param.get_conf_grid()
simulation.geom_param.compute_bxgradBoB2()
fz = np.maximum(-simulation.geom_param.bxgradBoB2[0,0,0,:],0)
plt.plot(z,fz,label=r'$\max(-G_{\nabla B}(x=x_{min},z),0)$')
plt.plot(z,eta2,label=r'$\eta_2(z), b=1.2$')
plt.plot(z,eta22,label=r'$\eta_2(z), b=1.5$')
plt.xlabel('z')
plt.legend();

In [ ]:
#---- Create a new gradB source that has an adapted amplitude 
#---- to match initial condition gradB losses
GBsource_adapted = copy.copy(GBsource_apriori) # creation of a new GBsource

# We first normalize our source by its volume integral
# setup auxiliary grids for trapezoidal integration
[x,y,z] = simulation.geom_param.get_conf_grid()
[X,Y,Z] = custom_meshgrid(x,y,z)
# evaluate the integrant on the mesh
integrant = GBsource_adapted.dens_source(X,Y,Z)*simulation.geom_param.Jacobian
# Compute the volume integral
integral = integral_xyz(x,y,z,integrant)
print(integral)
# Normalize the source amplitude by the integral
GBsource_adapted.nrate /= integral
# We can perform again the volume integral to verify that it is equal to one
integrant = GBsource_adapted.dens_source(X,Y,Z)*simulation.geom_param.Jacobian
integral = integral_xyz(x,y,z,integrant)
print(integral)
# We now scale the nrate to match the initial particle loss rate
# Profile initial conditions
def nT_IC(x, specie):
    n0 = specie.n0
    Te0 = specie.T0
    dens = n0 * (0.5 * (1 + np.tanh(3 * (0.1 - 10 * x))) + 0.01)
    Temp = 6 * Te0 * (0.5 * (1 + np.tanh(3 * (-0.1 - 10 * x))) + 0.01)
    return [dens,Temp]
# Get the inner radius IC value
[nx0, Tx0] = nT_IC(0,ion)
# Compute the initial loss rate


In [ ]:
#--- Compare the source and loss along time
plt.plot(time,[-g for g in GBloss_e_t],label=r'$-S_{\nabla B e, loss}$')
plt.plot(time,[-g for g in GBloss_i_t],label=r'$-S_{\nabla B i, loss}$')
plt.plot(time,np.abs(GBsrc_t),label=r'$|S_{\nabla B, source}|$',color='black',linestyle='--')
plt.ylabel(r'#particles/s')
plt.xlabel(r'$t$ ($\mu$s)')
plt.title('Particle loss at the inner flux surface')
plt.yscale('log')
plt.legend()